In [ ]:
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Chargement des données
def load_data_from_csv(file_path):
    df = pd.read_csv(file_path, sep=',', header=0)
    df.columns = ['timestamp', 'USD/GBP','USD/CAD','USD/EUR','USD/AUD','GBP/USD','GBP/CAD','GBP/EUR','GBP/AUD',
                  'CAD/USD','CAD/GBP','CAD/EUR','CAD/AUD','EUR/USD','EUR/GBP','EUR/CAD','EUR/AUD',
                  'AUD/USD','AUD/GBP','AUD/CAD','AUD/EUR']
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

# Liste des paires de devises
currency_pairs = [
    'USD/GBP', 'USD/CAD', 'USD/EUR', 'USD/AUD',
    'GBP/USD', 'GBP/CAD', 'GBP/EUR', 'GBP/AUD',
    'CAD/USD', 'CAD/GBP', 'CAD/EUR', 'CAD/AUD',
    'EUR/USD', 'EUR/GBP', 'EUR/CAD', 'EUR/AUD',
    'AUD/USD', 'AUD/GBP', 'AUD/CAD', 'AUD/EUR'
]

# Création de l'application Dash
app = dash.Dash(__name__, external_stylesheets=[
    "https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css",
    "https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;600&display=swap"
])

# Définition du layout
app.layout = html.Div(style={"backgroundColor": "#0a192f", "color": "#b23654", "fontFamily": "Montserrat"}, children=[
    html.H1("\ud83d\udcc8 FX Exchange Rates Dashboard", className="text-center mt-4", style={"fontWeight": "600"}),

    html.Div(className="container mt-3", children=[
        html.Label("Select Currency Pair:", className="form-label", style={"fontWeight": "400", "fontSize": "18px"}),
        dcc.Dropdown(
            id="currency-pair-dropdown",
            options=[{"label": pair, "value": pair} for pair in currency_pairs],
            value="USD/EUR",
            clearable=False,
            className="mb-3"
        ),
        dcc.Graph(id="currency-pair-graph")
    ]),

    html.Div(className="container mt-4", children=[
        html.Div(className="row justify-content-center", children=[
            html.Div(className="col-md-3", children=[
                html.Div(className="card text-white shadow-sm", style={"backgroundColor": "#005f73", "borderRadius": "10px"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.H5("High \ud83d\udcc8", className="card-title"),
                        html.H3(id="high-value", style={"fontWeight": "600"})
                    ])
                ])
            ]),
            html.Div(className="col-md-3", children=[
                html.Div(className="card text-white shadow-sm", style={"backgroundColor": "#ae2012", "borderRadius": "10px"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.H5("Low \ud83d\udcc9", className="card-title"),
                        html.H3(id="low-value", style={"fontWeight": "600"})
                    ])
                ])
            ]),
            html.Div(className="col-md-3", children=[
                html.Div(className="card text-white shadow-sm", style={"backgroundColor": "#ee9b00", "borderRadius": "10px"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.H5("Volatility 🚀 ", className="card-title"),
                        html.H3(id="volatility-value", style={"fontWeight": "600"})
                    ])
                ])
            ]),
             html.Div(className="col-md-3", children=[
                html.Div(className="card text-white shadow-sm", style={"backgroundColor": "#1f7a1f", "borderRadius": "10px"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.H5("Return (24h) 📊", className="card-title"),
                        html.H3(id="return-value", style={"fontWeight": "600"})
                    ])
                ])
            ])
        ])
    ]),

    dcc.Interval(
        id='interval-component',
        interval=5*60*1000,  # Rafraîchir toutes les 5 minutes
        n_intervals=0
    )
])

# Callback de mise à jour
@app.callback(
    [Output("currency-pair-graph", "figure"),
     Output("high-value", "children"),
     Output("low-value", "children"),
     Output("volatility-value", "children")],
    [Input("currency-pair-dropdown", "value"),
     Input("interval-component", "n_intervals")]
)
def update_dashboard(currency_pair, n_intervals):
    data = load_data_from_csv('/home/azureuser/scraping_dir/data/exchange_rates.csv')
    filtered_data = data[['timestamp', currency_pair]]

    fig = px.line(filtered_data, x="timestamp", y=currency_pair,
                  title=f"Exchange Rates for {currency_pair}",
                  labels={"value": "Exchange Rate", "timestamp": "Time"},
                  template="plotly_dark")

    high_rate = round(filtered_data[currency_pair].max(), 4)
    low_rate = round(filtered_data[currency_pair].min(), 4)
    volatility = round(filtered_data[currency_pair].std(), 4)

    return fig, high_rate, low_rate, volatility

# Exécution de l'application
if __name__ == "__main__":
    app.run_server(host="0.0.0.0", port=8050, debug=True)

SyntaxError: unmatched ']' (3171265392.py, line 3)

In [10]:
import datetime

def generate_daily_report(file_path):
    data = load_data_from_csv(file_path)

    # Filtrer les données de la journée actuelle
    today = datetime.datetime.now().date()
    daily_data = data[data['timestamp'].dt.date == today]

    if daily_data.empty:
        return "No data available for today."

    report = f"📊 Daily Report - {today}\n"
    
    for currency_pair in daily_data.columns[1:]:  # Exclure 'timestamp'
        open_price = daily_data[currency_pair].iloc[0]
        close_price = daily_data[currency_pair].iloc[-1]
        high_price = daily_data[currency_pair].max()
        low_price = daily_data[currency_pair].min()
        volatility = daily_data[currency_pair].std()
        daily_return = ((close_price - open_price) / open_price) * 100 if open_price != 0 else 0

        report += f"\n🔹 {currency_pair}:\n"
        report += f"   📈 Open: {open_price:.4f} | 📉 Close: {close_price:.4f}\n"
        report += f"   🔺 High: {high_price:.4f} | 🔻 Low: {low_price:.4f}\n"
        report += f"   ⚡️ Volatility: {volatility:.4f}\n"
        report += f"   📊 Return: {daily_return:.2f}%\n"

    # Sauvegarder le rapport dans un fichier
    with open("/home/azureuser/scraping_dir/daily_report.txt", "w") as f:
        f.write(report)

    return report

In [11]:
generate_daily_report('/Users/Dell/Downloads/exchange_rates.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/azureuser/scraping_dir/daily_report.txt'